In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import dataloader, TensorDataset

%matplotlib widget

## Memeriksa ketersedian CUDA
CUDA memanfaatkan komputasi pemrogramam paralel dengan menggunakan GPU

In [3]:
if torch.cuda.is_available():
    dev = 'cuda'
else:
    dev = 'cpu'

device = torch.device(dev)
print(f'Komputasi menggunakan {dev}...') 

Komputasi menggunakan cuda...


## Memuat Data
Memuat data, kemudian dibagi menjadi 3 bagian training set, validation set, test set.

In [11]:
# Memuat data Iris.csv
def get_data(path):
    path = '../machinel/data/Iris.csv'

    df = pd.read_csv(path)
    X = df.iloc[:,1:5].values
    X = torch.tensor(X, dtype=torch.float32)

    y = df.Species
    enc = LabelEncoder()
    y = enc.fit_transform(y)
    y = torch.from_numpy(y).type(torch.LongTensor)
    return X, y

# Membagi dataset menjadi 3 bagian: training, validation, dan test set.

def split_data(X, y, b_size):
    training, test_x, training_label, test_y = train_test_split(X,y, test_size=0.2, random_state=42)
    train_x, val_x, train_y, val_y = train_test_split(training, training_label, test_size=0.2, random_state=42)

    train = TensorDataset(train_x, train_y)
    val = TensorDataset(val_x, val_y)
    test = TensorDataset(test_x, test_y)

    train_loader = dataloader(train, batch_size=b_size)
    val_loader = dataloader(val, batch_size=b_size)
    test_loader = dataloader(test, batch_size=b_size)

    print(f'train size: {len(train_loader.dataset)}, 
          val size: {len(val_loader.dataset)}, 
          test size: {len(test_loader.dataset)}')
    print(f'train num bacthes: {len(train_loader)}, 
          val : {len(val_loader)}, 
          test : {len(test_loader)}')
    return train_loader, val_loader, test_loader

## Membangun arsitektur model LSTM

## Menampilkan Confusion Matrix

## Tahap Training

## Tahap Validasi dan Uji

## Main Program

In [13]:
# memuat data
path = '../machinel/data/Iris.csv'
X, y = get_data(path)


#split data
batch_size = 5
train, val, test = split_data(X, y, batch_size)
# memulai training
#training
#validation

#done

(tensor([[5.1000, 3.5000, 1.4000, 0.2000],
         [4.9000, 3.0000, 1.4000, 0.2000],
         [4.7000, 3.2000, 1.3000, 0.2000],
         [4.6000, 3.1000, 1.5000, 0.2000],
         [5.0000, 3.6000, 1.4000, 0.2000],
         [5.4000, 3.9000, 1.7000, 0.4000],
         [4.6000, 3.4000, 1.4000, 0.3000],
         [5.0000, 3.4000, 1.5000, 0.2000],
         [4.4000, 2.9000, 1.4000, 0.2000],
         [4.9000, 3.1000, 1.5000, 0.1000],
         [5.4000, 3.7000, 1.5000, 0.2000],
         [4.8000, 3.4000, 1.6000, 0.2000],
         [4.8000, 3.0000, 1.4000, 0.1000],
         [4.3000, 3.0000, 1.1000, 0.1000],
         [5.8000, 4.0000, 1.2000, 0.2000],
         [5.7000, 4.4000, 1.5000, 0.4000],
         [5.4000, 3.9000, 1.3000, 0.4000],
         [5.1000, 3.5000, 1.4000, 0.3000],
         [5.7000, 3.8000, 1.7000, 0.3000],
         [5.1000, 3.8000, 1.5000, 0.3000],
         [5.4000, 3.4000, 1.7000, 0.2000],
         [5.1000, 3.7000, 1.5000, 0.4000],
         [4.6000, 3.6000, 1.0000, 0.2000],
         [5

## Menampilkan plot untuk accuracy & loss dari model

## Melakukan pengujian dengan test set